# 001.03 // data processing

[convert annotations to txt files for keras implementation of YOLOv3]

In [ ]:
import os
import shutil
import pandas as pd

In [ ]:
# move labels to designated file

def move_files(path_in, path_out, file_type):

    filenames = sorted([f for f in os.listdir(path_in) if file_type in f])
    
    for f in filenames:
        shutil.move(f'{path_in}/{f}', f'{path_out}{f}')

In [ ]:
move_files('../data/images/', '../data/labels/', '.xml')

In [ ]:
# convert xml annotations into csv

!python xml_to_csv.py

In [ ]:
df = pd.read_csv('../data/labels/_annotations.csv')
df.head()

In [ ]:
# confirm 52 classes

classes = sorted(df['class'].unique())

ids = {}
i = 0

for c in classes:
    ids[c] = i
    i += 1
    
len(classes)

In [ ]:
# create classes document

classes_txt = open('../assets/classes.txt', 'w+')

[i for i in ids.keys()]

for i in ids.keys():
    classes_txt.write(f'{i}\n')

classes_txt.close()

In [ ]:
# if using grayscale images, change first line to: 
# df['file_path'] = [f'../data/images_gs/{i}' for i in df['filename']]

df['file_path'] = [f'../data/images/{i}' for i in df['filename']]

df['class_id'] = df['class'].map(ids)

df = df.sort_values(['filename', 'class_id']).reset_index(drop = True)

In [ ]:
box_features = ['file_path', 'xmin', 'ymin', 'xmax', 'ymax', 'class_id']

boxes = df[box_features].applymap(str)

for i in box_features:
    if box_features.index(i) in range(1, 5):
        boxes[i] = boxes[i] + ','
    else:
        boxes[i] = boxes[i] + ' '

In [ ]:
# create final annotations document

samples = boxes['file_path'].unique()
annot_txt = open('../assets/annotations.txt', 'w+')
box_format = []

for i in boxes.index:
    box_format.append([boxes.loc[i, c] for c in box_features])

box_format.append([''])

for i in range(len(samples)):
    while box_format[i][0] == box_format[i + 1][0]:
        [box_format[i].append(n) for n in box_format[i + 1][1:]]
        box_format.pop(i + 1)

for i in box_format[:-1]:
    annot_txt.write((''.join(i).strip() + '\n'))

annot_txt.close()